## Code for solving the game Lights Out, in O(n^2)

In [135]:
import numpy as np
import copy
import math
import time
import itertools
import queue

def create_puzzle(rows, cols):
    matrix = np.arange(rows * cols)
    matrix = matrix.reshape(rows, cols)
    matrix[:] = 0
    return matrix

def print_2d_matrix(matrix):
    print(matrix)

def perform_move(matrix, length, width, x, y):
    '''
    inputs:
        the x,y coordinate to invert
    perform_move - invert current tile, and the left, right, up, down tiles

    returns:
        nothing
    '''
    matrix[x][y] = not matrix[x][y]  # double check this
    if (x != 0):
        matrix[x - 1][y] = not matrix[x - 1][y]
    if (x != (width - 1)):
        matrix[x + 1][y] = not matrix[x + 1][y]
    if (y != 0):
        matrix[x][y - 1] = not matrix[x][y - 1]
    if (y != (length - 1)):
        matrix[x][y + 1] = not matrix[x][y + 1]
        
def is_solved(matrix):
    '''
    checks if the game is solved
    returns:
        False if an entry is 1
    '''
    is_solved = True
    for col in matrix:
        for row in col:
            if (row == 1):
                return False
    return is_solved

def next_cell(x, y, width):
    if (x == width - 1):
        x = 0
        if (y == width - 1):
            return None
        else:
            y = y + 1
    else:
        x = x + 1
    return [x,y]

In [57]:
def get_upper_triangular(A,n,m):
    '''
    input:
    A - n by n matrix
    output:
    matrix A in upper triangular form
    Modified from regular gaussian elimination algorithm
    '''

    # expected solution: x1 = 1, x2 = 0, x3 = 1
    
    # Make deep copy of A
    
    #n = 3
    #m = 3
    if n != m:
        return
    
    # pivot column
    pivot_col = 0
    
    # Iterate through rows until the pivot col entry is 1
    for pivot_col in range(0,m): # m+1?
        max_i = pivot_col
        
        for row in range(pivot_col,n):

            if A[row,pivot_col] == 1: 
                max_i = row
                break

        if A[max_i,pivot_col] == 1:
            # Swap rows i and max_i
            A[[pivot_col, max_i]] = A[[max_i, pivot_col]]
            
            for u in range(pivot_col+1,m):
                if A[u][pivot_col] == 1:
                    A[u] = (A[pivot_col]+A[u])%2
        else:
            # more than one solution
            print("error")
    return A

def back_sub(A,n,m):
    '''
    A an n by m matrix, in upper triangular form
    '''

    x = np.zeros((n,), dtype=int)
    for i in range(n-1,-1,-1):
        if A[i,i] == 1:
            x[i] = A[i,n]
            for j in range(i+1,n):
                x[i] = (x[i] - x[j]*A[i][j]) % 2
        else:
            print("error :(")
    return x
    

def is_upper_triangular(A, n, m):
    '''
    A an n by m matrix, in upper triangular form
    '''
    A = np.array([[1, 0, 0, 1], [0, 1, 1, 0], [0, 0, 1, 1]])
    n = 3
    m = 3
    min = 0
    for row in A[1:]:
        for i in range(0,n):
            if row[i] == 1 and i > min:
                min = i
                break
            elif i <= min and row[i] == 1:
                return False
            elif row[i] != 0:
                return False
    return True

# Test the functions here

#A = np.array([[1, 0, 0, 1], [0, 1, 1, 0], [0, 0, 1, 1]])
A = np.array([[0,1,1,1], [1,0,0,1], [0,1,0,0]])
n = 3
m = 3

A_tri = get_upper_triangular(A,n,m)
    
if is_upper_triangular(A_tri,n,m):
    x = back_sub(A,n,m)
    print("Solution [x1,x2,x3] =" , x)

Solution [x1,x2,x3] = [1 0 1]


In [84]:
def get_game_vector(game_matrix):
    '''
    Creates a vector such that each element is 1 if the corresponding square is on, otherwise 0
    '''
    A = game_matrix.flatten('C')
    return A
    
A = np.array([[0,1,1,1], [1,0,0,1], [0,1,0,0]])
x = get_game_vector(A)
print(x)

[0 1 1 1 1 0 0 1 0 1 0 0]


In [ ]:
def get_matrix_from_game(n,m):
    '''
    Create n^2 x n^2 matrix for solving the game
    '''
    
    mat = np.zeros((n*n,n*n),dtype=int)

    for i in range(0,n*n):
        x = math.floor(i/n)
        y = (i)%n

        # Light up square

        mat[i][i] = 1

        if i%n != 0:
            # not left edge
            mat[i-1][i] = 1

        if i%n != n-1:
            # not right edge
            mat[i+1][i] = 1
                
        if i >= n:
            # Not top edge
            mat[i-n][i] = 1
                
        if i < n*(n-1):
            # Not bottom edge
            mat[i+n][i] = 1
    return mat

## Test get_matrix_for_game() with a 2x2 and a 3x3 game matrix

In [133]:
num_rows = 2
A = get_matrix_from_game(num_rows)
print("result:\n", A, "\n")

A = get_matrix_from_game(3)
print("result:\n", A)

result:
 [[1 1 1 0]
 [1 1 0 1]
 [1 0 1 1]
 [0 1 1 1]] 

result:
 [[1 1 0 1 0 0 0 0 0]
 [1 1 1 0 1 0 0 0 0]
 [0 1 1 0 0 1 0 0 0]
 [1 0 0 1 1 0 1 0 0]
 [0 1 0 1 1 1 0 1 0]
 [0 0 1 0 1 1 0 0 1]
 [0 0 0 1 0 0 1 1 0]
 [0 0 0 0 1 0 1 1 1]
 [0 0 0 0 0 1 0 1 1]]


In [ ]:
class Game:
    def create_game_matrix(self):
        return create_puzzle()

    def __init__(self, length, width):
        self.length = length
        self.width = width
        self.matrix = create_puzzle(length, width)
        self.moves = create_puzzle(length, width)
        self.solver = Game_solver(length, width) # new

    def init_move_matrix(self):
        '''
        init_move_matrix - create a zeros matrix to store moves
        '''
        for i in range(0, self.width):
            for j in range(0, self.length):
                rand_value = np.random.choice([0, 1])
                self.moves[i][j] = rand_value

    def scramble(self):
        '''
        scramble - randomly generate a 1 or 0 value for each tile in the grid
        add the value to the corresponding tiles.
        '''
        self.init_move_matrix()
        for x in range(0,self.length):
            for y in range(0,self.width):
                if (self.moves[x][y] == 1):
                     perform_move(self.matrix,self.length, self.width, x, y)

    def next_cell(self,row,col):
        next_row = row + 1
        next_col = col + 1
        if (row == self.width - 2):
            next_row = 0
        if (col == self.length - 2):
            next_col = 0
        return [next_row, next_col]

In [ ]:
class Game_solver:
    
    def __init__(self):
        '''
        something
        '''
        get_matrix_from_game(length, width)

    def solve(self):
        '''
        return all moves to solve the game
        '''
        return
        
    def get_hint(self):
        '''
        return one of the moves to solve the game
        '''
        return

In [ ]:
class Stack:
    '''
    This Stack class was taken from:
    http://interactivepython.org/runestone/static/pythonds/BasicDS/ImplementingaStackinPython.html
    '''
    def __init__(self):  
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def peek(self):
        return self.items[len(self.items)-1]

    def size(self):
         return len(self.items)

In [ ]:
def bitcount(n):
    '''
    Adapted from:
    https://stackoverflow.com/questions/8871204/count-number-of-1s-in-binary-representation
    '''
    count = 0
    for idx in range(0,len(n)):
        count = count + int(n[idx])
    return count

def bitcount_np(m, width):
    '''
    Adapted from:
    https://stackoverflow.com/questions/8871204/count-number-of-1s-in-binary-representation
    '''
    count = 0
    for idx in range (0,(width -1)):
        for j in range (0,(width -1)):
            count = count + int(m[idx][j])
    return count

In [ ]:
def dfs(puzzle, width):
    if not width:
        return
    n = width*width
    dfs_stack = Stack()
    matrix = copy.deepcopy(puzzle)
    
    # Create a stack
    for i in range(2**n,0,-1):
        value = bin(i)[2:]
        value = "0" * (n-len(value)) + value
        dfs_stack.push(value)
    
    # After an iteration, store the move that was previously tried
    prev_move = [0]* n
    
    # Pop items and test
    counter = 0
    for i in range (0, dfs_stack.size()):
        move = dfs_stack.pop()
        for pos in range (len(move),0,-1):
            if ((int(move[pos-1])+int(prev_move[pos-1])%2) == 1):
                row = math.floor((pos-1)/width)
                col = pos - (row)*(width) - 1
                perform_move(matrix,width,width,row,col)
                counter = counter + 1
        if (is_solved(matrix)):
            bits = bitcount(move)
            return [bits, counter]
        else:
            prev_move = move
    return

In [136]:
def main():
    print("Starting Lights Out!\n")
    '''
    Initialize the counters for 100 random puzzles
    '''
    length = 3
    width = length

    bfs_total_steps = 0
    dfs_total_steps = 0
    a_total_steps = 0
    
    bfs_min_steps = 0
    dfs_min_steps = 0
    a_min_steps = 0
    
    bfs_time = 0
    dfs_time = 0
    a_time = 0
    
    '''
    Loop through 100 random puzzles
    '''
    num_puzzles = 2
    for tests in range (0,num_puzzles):
        # Initialize a random matrix
        game = Game(length, width)
        game.scramble()
        print("Solve the puzzle:")
        print(game.matrix)
        
        # Perform 100 Depth First Search
        
        '''
        dfs_result = dfs(game.matrix, length)
        dfs_min_steps = dfs_min_steps + dfs_result[0]
        dfs_total_steps = dfs_total_steps + dfs_result[1]
        '''

    print("")

if __name__ == '__main__':
    main()

Starting Lights Out!

Solve the puzzle:
[[1 0 1]
 [1 0 1]
 [1 0 0]]
Solve the puzzle:
[[1 0 0]
 [1 1 1]
 [1 0 1]]

Total number of steps used 100 times
bfs: 0
dfs: 1393
a star: 0

Minimum number of steps in the solutions, 100 times
bfs: 0
dfs: 11
a star: 0

Total length of time for 100 algorithms
bfs: 0 seconds
dfs: 0 seconds
a star: 0 seconds
The end.
